In [1]:
import os
import shutil
import time
import pandas as pd
from tqdm.notebook import tqdm
import re
from pathlib import Path
import traceback
import file_function as ff

In [22]:
def file_info(path):

    dict_size = {}  # 중복파일명 숫자를 카운트 할 딕셔너리

    cols = ['size', 'mtime', 'sell', 'root', 'stem', 'length', 'ext']
    df = pd.DataFrame(columns=cols)
    
    df_matching = pd.read_excel(
        r'C:\Users\SL\OneDrive\workspace\company\fileNaming\파일\매각사 이름매칭.xlsx')

    for root, __dirs__, files in tqdm(os.walk(path)):
        
        for f in files:
            fullname = os.path.join(root, f)
            #key
            size = os.path.getsize(fullname)
            #value
            mtime = os.path.getmtime(fullname)
            sell = ""
            for __index__, row in df_matching.iterrows():
                if re.search(str(row[0]), str(root)):  
                    sell = row[1]
                    break            
            stem = os.path.splitext(f)[0]
            length = len(stem)
            ext = os.path.splitext(f)[1]
            
            temp1 = {"size":size}
            temp2 = {"mtime":mtime, "sell":sell, "root":root, "stem":stem, "length":length, "ext":ext}
            
            # 고유한 size를 키로 하는 딕셔너리 만들기
            if size not in dict_size:
                dict_size[size] = [temp2]
            else:
                dict_size[size].append(temp2)
            
            # size도 하나의 col로 df 만들기
            temp1.update(temp2)
            temp_df = pd.DataFrame([temp1])
            
            df = pd.concat([df, temp_df])
    
    df.to_excel(r"C:\Users\SL\Desktop\파일정보.xlsx")
            
    return dict_size

In [25]:
path = r'\\192.168.10.43\공유폴더\@◆통합파일◆@'
dict_size = file_info(path)

0it [00:00, ?it/s]

In [26]:
dict_size

{132051: [{'mtime': 1538723390.683,
   'sell': '디엔피에이엠씨대부㈜',
   'root': '\\\\192.168.10.43\\공유폴더\\@◆통합파일◆@\\(1.DNP_헬프)\\-이름, 사건번호 안뜸',
   'stem': '김향숙 2018카기866',
   'length': 13,
   'ext': '.pdf'}],
 96126: [{'mtime': 1561085378.7169974,
   'sell': '디엔피에이엠씨대부㈜',
   'root': '\\\\192.168.10.43\\공유폴더\\@◆통합파일◆@\\(1.DNP_헬프)\\-이름, 사건번호 안뜸',
   'stem': '김현지 2019타기190',
   'length': 13,
   'ext': '.pdf'}],
 74528: [{'mtime': 1571041091.5318708,
   'sell': '디엔피에이엠씨대부㈜',
   'root': '\\\\192.168.10.43\\공유폴더\\@◆통합파일◆@\\(1.DNP_헬프)\\-이름, 사건번호 안뜸',
   'stem': '나신기_양통',
   'length': 6,
   'ext': '.pdf'},
  {'mtime': 1536277645.7144,
   'sell': '디엔피에이엠씨대부㈜',
   'root': '\\\\192.168.10.43\\공유폴더\\@◆통합파일◆@\\(1.DNP_헬프)\\3.채권양도통지서',
   'stem': '20417766 전영호_양통',
   'length': 15,
   'ext': '.pdf'},
  {'mtime': 1536277645.7144,
   'sell': '디엔피에이엠씨대부㈜',
   'root': '\\\\192.168.10.43\\공유폴더\\@◆통합파일◆@\\(1.DNP_헬프)\\3.채권양도통지서\\디앤피양도통지서',
   'stem': '(주)쿠키아카데미의 보증인 전영호',
   'length': 18,
   'ext': '.pdf'}],
 48188:

In [33]:
os.chdir('c:/')

In [2]:
#### 최종확인 1
path = r'C:\Users\SL\Desktop\test'
ff.final_rename(path)

  0%|          | 0/5 [00:00<?, ?it/s]

5 개의 파일 이름 변경 완료
error : 
파일명 변경 목록 : 
['20407822_김대식_초본_(1).pdf', '20407822_김대식_초본_(2).pdf']
['20447360_ALSERRI_HAMED_OMAR_ABDULQADER_외국인증명.pdf', '20447360_ALSERRIHAMEDOMARABDULQADER_외국인증명.pdf']


In [64]:
#[시작]f[2]:문서종류만 이상없는지 살피기 ####################################################
p_docu = re.compile('원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')

path = r'D:\부산 정리'
os.chdir(path)
file_list = os.walk(path)
error_short = []
error_docu = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            splitname = f.split("_")
            if len(splitname)<3 :
                error_short.append(f)
            elif p_docu.search(splitname[2]) == None :
                error_docu.append(f)
print("short :", *error_short, sep="\n")
print("docu :", *error_docu, sep="\n")
#[끝]문서종류만 이상없는지 살피기 ####################################################

0it [00:00, ?it/s]

short :
20459630ZONG WEI_외국인증명.pdf
docu :


In [38]:
#[시작]채무자키로 시작하는지만 살피기 ####################################################
p_key = re.compile('[0-9]{8}')
p_notkey = re.compile('[0-9]{9,}')

path = r'D:\부산 정리'
os.chdir(path)
file_list = os.walk(path)
error = []

for r, d, files in tqdm(file_list) :
    for f in files :
        if f != "Thumbs.db":
            if p_key.match(f) == None :
                error.append(f)
            if p_notkey.match(f) :
                error.append(f)
            
print(error)
#[끝]문서종류만 이상없는지 살피기 ####################################################

0it [00:00, ?it/s]

[]


In [59]:
#[시작]이름에 있는 _ 를 공백으로 바꾸기 ####################################################
### 등,초본이 초본을 포함해버린다. 이거 수정해야
path = r'D:\부산 정리'

p_docu = re.compile(
    '원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명')
p_ = re.compile("_")

os.chdir(path)
file_list = os.walk(path)
fixed = []
count = 0

for r, d, files in tqdm(file_list):
    for f in files:
        if f != "Thumbs.db":
            new_name = ""
            if p_docu.search(f):
                st = p_docu.search(f).start() - 1
                new_name = f[:9] + p_.sub(" ", f[9:st]) + f[st:]  # 앞뒤 언더바 말고 중간에 언더바

                if f != new_name:
                    os.rename(r+"/"+f, r+"/"+new_name)
                    count += 1
                    fixed.append([f, new_name])
                    

print(count, "개의 파일 이름 변경 완료")
print(*fixed, sep="\n")

#[끝]####################################################

0it [00:00, ?it/s]

0 개의 파일 이름 변경 완료



In [58]:
#[시작]특정 단어(표현식) 포함하고 있는지 ####################################################
path = r'D:\부산 정리'
p1 = re.compile('__')

os.chdir(path)
file_list = os.walk(path)
fix = []
count = 0

for r, d, files in tqdm(file_list):
    for f in files:
        if f != "Thumbs.db":
            new_name = ""
            if p1.search(f) :
                fix.append([f, new_name])
                    

print(count, "개의 파일 이름 변경 완료")
print(*fixed, sep="\n")
#[끝]####################################################

0it [00:00, ?it/s]

0 개의 파일 이름 변경 완료



In [86]:
#[시작]외국인들 전산에서 이름정보 불러와서 rename ####################################################
# 파일명에 보증인 키워드가 있는지만 보고 그거에 맞춰서 ... 
path = r'D:\부산 정리'
os.chdir(path)
file_list = os.walk(path)

p_eng = re.compile("[a-zA-Z]+")
p_etc = re.compile("보증인")


count = 0

for root, dir, files in tqdm(file_list) : 
    for f in files : 

        if f != "Thumbs.db":
            new_name = ""
            
            spl_name = f.split("_", maxsplit=2)

            key = spl_name[0]
            debtor_name = spl_name[1]
            others = spl_name[2]

            if p_eng.search(debtor_name) : # 이름에 영어 있는 경우
                if p_etc.search(others) : # 보증인 키워드 있는 경우
                    new_name = key +"_" + str(dict[key].보증인성명)+"_"+ others
                else :
                    new_name = key +"_" + str(dict[key].성명)+"_"+others
                    
                if new_name != f :
                    src = os.path.join(root, f)
                    dst = os.path.join(root, new_name)
                    ff.rename(src, dst)
                    count += 1
print(count, "개의 파일명 변경 완료")
                  

0it [00:00, ?it/s]

20412275_LE THANH HAI_외국인증명.pdf 20412275_레탕하이_외국인증명.pdf
20432140_YANGXUEHUI_원인서류.pdf 20432140_YANG XUEHUI_원인서류.pdf
20432235_PIXIANGXU_원인서류.pdf 20432235_PI XIANGXU_원인서류.pdf
20432249_THOMASWHALLEY_원인서류.pdf 20432249_THOMAS WHALLEY_원인서류.pdf
20432299_LICHUNHUA_원인서류.pdf 20432299_LI CHUNHUA_원인서류.pdf
20432493_YIJAERAN(이제란)_판결문.pdf 20432493_YI JAE RAN_판결문.pdf
20432493_YIJAERAN_원인서류.pdf 20432493_YI JAE RAN_원인서류.pdf
20432503_SPOHRRYANAUSTIN(조성수)_판결문.pdf 20432503_SPOHR RYAN AUSTIN_판결문.pdf
20432503_SPOHRRYANAUSTIN_원인서류.pdf 20432503_SPOHR RYAN AUSTIN_원인서류.pdf
20432517_XUJI_원인서류.pdf 20432517_XU JI_원인서류.pdf
20432533_KIMANTHONYJONGUK_원인서류.pdf 20432533_KIM ANTHONY JONGUK_원인서류.pdf
20432543_LIANGRENLONG_원인서류.pdf 20432543_LIANG RENLONG_원인서류.pdf
20432551_SAMCHANTHOL_원인서류.pdf 20432551_SAM CHANTHOL_원인서류.pdf
20432560_PIAOXIUYU_원인서류.pdf 20432560_PIAO XIUYU_원인서류.pdf
20432623_LINSHENGLONG_원인서류.pdf 20432623_LIN SHENGLONG_원인서류.pdf
20432624_KIMNATALIA_원인서류.pdf 20432624_KIM NATALIA_원인서류.pdf
20432677_PAIKSEUNGHO_원인서류.

In [74]:
dst = r"C:\Users\SL\Desktop\test\새 폴더\20119538_임 신호_초본 (1).pdf"
dir = os.path.split(dst)[0]
f_name = os.path.split(dst)[1]
stem = os.path.splitext(f_name)[0]
ext = os.path.splitext(f_name)[1]
new_name = re.sub("[^가-힣]+$", "", stem)
print(*[dir, f_name, stem, ext, new_name], sep="\n")

C:\Users\SL\Desktop\test\새 폴더
20119538_임 신호_초본 (1).pdf
20119538_임 신호_초본 (1)
.pdf
20119538_임 신호_초본


C:\Users\SL\Desktop\test\새 폴더
20119538_임 신호_초본.pdf
20119538_임 신호_초본
.pdf


In [9]:
# 보충작업 : 1) 이름 엑셀문서에서 불러오기. 보증인 있으면 보증인 이름으로
path = r''
os.chdir(path)
file_list = os.listdir(path)

docu_kind = '원인서류|채권양도통지서|판결문|지급명령|이행권고|화해권고|타채|결정문|등본|초본|개회|신복|파산|외국인증명'
error = []

p_docu = re.compile(docu_kind)

for f in tqdm(file_list) : 
    if os.path.isfile(f) & (f != "Thumbs.db"):
        key = f[:8]
        docu = p_docu.search(f)
        if docu == None :
            error.append(f)
            continue
        else :
            refer = df_c[df_c.채무자키 == key]


        

numpy.int64

In [2]:
#### 최종확인 2
path = r'D:\부산 정리\KB국민카드'
ff.final_check(path)

  0%|          | 0/42746 [00:00<?, ?it/s]

70 개의 파일 처리 완료
메인3개
20412275_LE_THANH_HAI_외국인증명.pdf
20412295_이미향_등,초본.pdf
20412332_손보권_등,초본.pdf
20412392_박재현_등,초본.pdf
20412424_한국현_등,초본.pdf
20412451_이주봉_등,초본.pdf
20432140_YANG_XUEHUI_외국인증명.pdf
20432517_XU_JI_외국인증명.pdf
20433626_MOON_KYEOUNG_DON_외국인증명.pdf
20433626_MOON_KYEOUNG_DON_외국인증명_(1).pdf
20440157_LIJINHAI_외국인증명.pdf
20440157_LIJINHAI_외국인증명_(1).pdf
20445989_사공_진_초본.pdf
20446083윤정희_등본.pdf
20446182_정승우_등분.pdf
20446424_이진혁_등,초본.pdf
20446675_판결문_재도.pdf
20446971_GAO_YINGYONG_외국인증명.pdf
20446971_GAO_YINGYONG_외국인증명_(1).pdf
20447075_NANMINGZI_외국인증명.pdf
20447075_NAN_MINGZI_외국인증명.pdf
20447123_JIANG_LAN_외국인증명.pdf
20447123_JIANG_LAN_외국인증명_(1).pdf
20447292_RAHAYU_SRI_외국인증명.pdf
20447292_RAHAYU_SRI_외국인증명_(1).pdf
20447300_TURAEV_SHUHRAT_외국인증명.pdf
20447307_JIN_JINGHUA_외국인증명.pdf
20447360_ALSERRI_HAMED_OMAR_ABDULQADER_외국인증명.pdf
20447360_AL_SERRIHAMED_외국인증명.pdf
20447464_SOE_YU_YU_HNIN_외국인증명.pdf
20447471_VOKHIDOV_NEMATILLO_외국인증명.pdf
20447484_KORUWAKANKANAMGE_외국인증명.pdf
20447484_MAHESH_GAYANFERNANDO_외국인증명.

### 모든 정보 파일에 매각사 추가하기
- file_function에 추가 완료

In [ ]:
## 최초 df 불러오기, 참조매각
df_s = pd.read_csv(r'C:\Users\SL\Desktop\부산 모든 파일 정보.csv', encoding='euc-kr', index_col=0)


In [ ]:

## 매각사 칼럼 추가하기
df_matching = pd.read_excel(r'C:\Users\SL\Desktop\부산 매각사 이름매칭.xlsx')

sell = []

for i in df_s.경로 : 
    for index, row in df_matching.iterrows() :
        if re.search(row[0], i) : 
            sell.append(row[1])
            continue

In [ ]:
df_s['매각사'] = sell
df_s
    

,경로,파일명,중복수,크기,생성일,수정일,확장자,매각사
0,C:/Users/SL/Downloads/부산 공유폴더\0. DNP_헬프\10. 초본,20407803_이형구_초본_2020.10.23.pdf,1,2031607,2022.07.05 - 14:52:19,2020.10.23 - 16:56:39,.pdf,디엔피에이엠씨대부㈜
1,C:/Users/SL/Downloads/부산 공유폴더\0. DNP_헬프\10. 초본,20408088_김혜성_초본_2021.01.18.pdf,1,1455297,2022.07.05 - 14:52:19,2021.01.18 - 16:00:25,.pdf,디엔피에이엠씨대부㈜
2,C:/Users/SL/Downloads/부산 공유폴더\0. DNP_헬프\10. 초본,20408405_박성룡_초본_2020.11.13.pdf,1,1077317,2022.07.05 - 14:52:19,2020.11.13 - 14:06:44,.pdf,디엔피에이엠씨대부㈜
3,C:/Users/SL/Downloads/부산 공유폴더\0. DNP_헬프\10. 초본,20411188_임종우_초본_2021.05.07.pdf,1,814947,2022.07.05 - 14:52:19,2021.05.07 - 13:30:44,.pdf,디엔피에이엠씨대부㈜
4,C:/Users/SL/Downloads/부산 공유폴더\0. DNP_헬프\10. 초본,20411194_김정하_초본_2020.10.21.pdf,1,2569639,2022.07.05 - 14:52:19,2020.10.21 - 16:30:19,.pdf,디엔피에이엠씨대부㈜
...,...,...,...,...,...,...,...,...
72412,C:/Users/SL/Downloads/부산 공유폴더\7. 전북은행\3. 채권양도통...,20496704_박수묵_양도통지서.pdf,1,469523,2022.07.05 - 15:01:45,2022.05.19 - 15:38:05,.pdf,㈜전북은행
72413,C:/Users/SL/Downloads/부산 공유폴더\7. 전북은행\3. 채권양도통...,S28BW-5e22051915400_0027.pdf,1,471395,2022.07.05 - 15:01:45,2022.05.19 - 15:38:11,.pdf,㈜전북은행
72414,C:/Users/SL/Downloads/부산 공유폴더\7. 전북은행\3. 채권양도통...,S28BW-5e22051915400_0028.pdf,1,509887,2022.07.05 - 15:01:45,2022.05.19 - 15:38:12,.pdf,㈜전북은행
72415,C:/Users/SL/Downloads/부산 공유폴더\8. 베리타스\양수도계약서,베리타스_양수도_계약서.pdf,1,31019960,2022.07.05 - 15:01:46,2022.07.01 - 18:04:05,.pdf,㈜베리타스자산대부


### 채무자키 집어넣기

In [59]:
## 모든 파일 정보 불러오기
df_s = ff.all_files(r'C:\Users\SL\Downloads\부산 공유폴더', '부산')
df_s.head()

541it [00:20, 26.44it/s]


,경로,매각사,파일명,중복수,크기,생성일,수정일,확장자
0,C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본,디엔피에이엠씨대부㈜,20407803_이형구_초본_2020.10.23.pdf,1,2031607,2022.07.05 - 14:52:19,2020.10.23 - 16:56:39,.pdf
1,C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본,디엔피에이엠씨대부㈜,20408088_김혜성_초본_2021.01.18.pdf,1,1455297,2022.07.05 - 14:52:19,2021.01.18 - 16:00:25,.pdf
2,C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본,디엔피에이엠씨대부㈜,20408405_박성룡_초본_2020.11.13.pdf,1,1077317,2022.07.05 - 14:52:19,2020.11.13 - 14:06:44,.pdf
3,C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본,디엔피에이엠씨대부㈜,20411188_임종우_초본_2021.05.07.pdf,1,814947,2022.07.05 - 14:52:19,2021.05.07 - 13:30:44,.pdf
4,C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본,디엔피에이엠씨대부㈜,20411194_김정하_초본_2020.10.21.pdf,1,2569639,2022.07.05 - 14:52:19,2020.10.21 - 16:30:19,.pdf


In [124]:
## 참조 df 불러오기
## 참조할 col_name = '성명', '주민번호', '채무자키', '매각사', '보증인성명'
df_c = pd.read_excel(r'C:\Users\SL\Desktop\채무자조회.xlsx')
df_c.head()

c:\Users\SL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,채무자키,성명,주민번호,성별,연령,매각사,변경전성명,변경전주민번호
0,20495151,(유)가람건설,403-81-18275,법인,82,㈜전북은행,NaN,NaN
1,20495152,(유)가람건설,403-81-18275,법인,82,㈜전북은행,NaN,NaN
2,20495238,(유)가람건설,403-81-18275,법인,82,㈜전북은행,NaN,NaN
3,20417854,(유)가람관광여행사,403-81-30809,법인,81,디엔피에이엠씨대부㈜,NaN,NaN
4,20411449,(유)거성산업,407-81-06562,법인,81,디엔피에이엠씨대부㈜,NaN,NaN


In [152]:
## 1) 채무자키 없는 경우 찾기 - 모든 파일 정보 파일명에서 숫자 8자리이상 있고, 9자리 이상은 없는 경우
p_n8 = re.compile('[0-9]{8}') # 8자리 이상 연속 숫자(채무자키 + 관리자키)
p_n9 = re.compile('[0-9]{9}')
p_n10 = re.compile('[0-9]{10}') # 10자리 이상(관리자키 + ?) 9자리가 하나 있네?
p_n11 = re.compile('[0-9]{11}') 
p_n12 = re.compile('[0-9]{12}')
p_admin = re.compile('^\d\d\d\d-\d\d\d\d\d\d\d|_\d\d\d\d-\d\d\d\d\d\d\d')
# Match함수(match, search)는 Match객체를 반환하는데, 표현식과 일치하는 부분의 시작인덱스와, 
# 끝인덱스, 해당문자열을 가지고있다.
# match는 시작부분에 있어야 하고, search는 중간에 있어도 찾는다. 단 만족하는 값을 찾으면 추가검색은 하지 않는다.
# findall은 겹치지 않게 모두를 검색한다. ex) ololo -> olo를 찾고, 나머지 lo에서 olo가 있는지 또 찾는다.


In [128]:
# 채무자키 없는 거만 추려서 새df : df_s_noKey로
df_s_noKey = pd.DataFrame()
ex = len(df_s.index)

for index, row in df_s.iterrows() :
    s8 = p_n8.search(row.파일명)
    s9 = p_n9.search(row.파일명)
    if not (s8 != None) & ( s9== None) :
           #__________________________
     # not    채무자키 있는 거
        df_s_noKey = pd.concat([df_s_noKey, df_s[index:index+1]], ignore_index=True)
        

print("총 "+str(ex) + "개 중, 채무자키 없는 것(df_s_noKey) : " + str(len(df_s_noKey.index)))
print(df_s_noKey.head())
print(df_s_noKey.tail())

72362it [01:29, 809.97it/s]  

총 72362개 중, 채무자키 없는 것(df_s_noKey) : 54676
                                                  경로      매각사  \
0  C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서...  한울가람대부㈜   
1  C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서...  한울가람대부㈜   
2  C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서...  한울가람대부㈜   
3  C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서...  한울가람대부㈜   
4  C:\Users\SL\Downloads\부산 공유폴더\1.한울가람_헬프\2. 원인서...  한울가람대부㈜   

                                        파일명  중복수       크기  \
0                             1698_일진산기.pdf    1  6830222   
1  1699_항진무역(주)_7001000032_120-81-02335.pdf    1  4058466   
2                              1699항진무역.pdf    1  4058466   
3  1700_신도광학(주)_7001000180_301-81-14556.pdf    1  3973249   
4                             1700_신도광학.pdf    1  3973249   

                     생성일                    수정일   확장자  
0  2022.07.05 - 14:52:22  2020.10.19 - 14:03:58  .pdf  
1  2022.07.05 - 14:52:22  2020.10.19 - 14:19:48  .pdf  
2  2022.07.05 - 14:52:22  2020.1

In [158]:
# 관리자키가 파일명에 들어가 있는 경우의 파일명을 확인해보자.
df_temp = pd.DataFrame()
for index, row in df_s.iterrows() :
    s10 = p_n10.search(row.파일명)
    s12 = p_n12.search(row.파일명)
    s_admin1 = (s10 != None) & (s12 == None)
    s_admin2 = (p_admin.search(row.파일명) != None)
    
    # 관리자키인 경우 df_temp에 추가
    
    if s_admin1 | s_admin2 : 
        df_temp = pd.concat([df_temp, df_s[index : index+1]], ignore_index=True)

LookupError: unknown encoding: utf-sig-8

In [159]:
df_temp.to_csv(r'C:\Users\SL\Desktop\관리자키있는거.csv', encoding='utf-8-sig')

In [126]:
## 2) df_s_noKey에서 관리자 키가 있는 경우(10자리,11자리 == 10이상 & 12자리이상x)
# 전체조회 결과 관리자키는 1)없거나, 2)10자리거나, 3)11자리거나, 4)4자리-7자리
for index, row in df_s_noKey.iterrows() :
    s10 = p_n10.search(row.파일명)
    s12 = p_n12.search(row.파일명)
    if (s10 != None) & (s12== None) :
#블라블라





72362it [00:20, 3470.21it/s] 

총 72362개 중, 채무자키 없는 행 : 20803
                                               경로         매각사  \
0  C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본  디엔피에이엠씨대부㈜   
1  C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본  디엔피에이엠씨대부㈜   
2  C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본  디엔피에이엠씨대부㈜   
3  C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본  디엔피에이엠씨대부㈜   
4  C:\Users\SL\Downloads\부산 공유폴더\0. DNP_헬프\10. 초본  디엔피에이엠씨대부㈜   

                              파일명  중복수       크기                    생성일  \
0  20407803_이형구_초본_2020.10.23.pdf    1  2031607  2022.07.05 - 14:52:19   
1  20408088_김혜성_초본_2021.01.18.pdf    1  1455297  2022.07.05 - 14:52:19   
2  20408405_박성룡_초본_2020.11.13.pdf    1  1077317  2022.07.05 - 14:52:19   
3  20411188_임종우_초본_2021.05.07.pdf    1   814947  2022.07.05 - 14:52:19   
4  20411194_김정하_초본_2020.10.21.pdf    1  2569639  2022.07.05 - 14:52:19   

                     수정일   확장자  
0  2020.10.23 - 16:56:39  .pdf  
1  2021.01.18 - 16:00:25  .pdf  
2  2020.11.13 - 14:06:44  .pdf  
3 

In [123]:
a = "가나다_9014000712_가나다_라마"
s10 = p_n10.search(a)
s11 = p_n11.search(a)
if (s10!= None) & ( s11== None) :

# df_c에서 관리자키를 참조하여 채무자키 확보
    for index, row in df_c.iterrows() :
        if row.

        s10.group()
        re.sub(s10.group(),"",a)


(4, 14)
1234567891
가나다__가나다_라마


In [151]:
a = '34_1234-1234567_가나다'
p_admin = re.compile('^\d\d\d\d-\d\d\d\d\d\d\d|_\d\d\d\d-\d\d\d\d\d\d\d')
re.search(p_admin, a).group()

'_1234-1234567'

In [28]:
# 1) 관리자키 검색 : 10자리이상 11자리 none
for index, row in df_s_noKey.iterrows() :
    if (p_n10.match(row.파일명) != None) & (p_n11.match(row.파일명) == None) :
        
print(채무자키)

# 2) df_c에서 관리자키를 참조하여 채무자키 확보
# 3) 확보한 채무자키를 파일명에 넣고, 관리자키는 파일명에서 삭제(rename)
# 4) df_s의 파일명도 수정 or '모든파일정보'함수 재실행


,파일명
0,12345678
1,123456789


In [24]:
## 관리자키 있는 경우 보기

for index, row in df_s.iterrows :
    if (p_n8.search(row.파일명) != None) & (p_n9.search(row.파일명) == None) :
        print(row.파일명)

0        20407803_이형구_초본_2020.10.23.pdf
1        20408088_김혜성_초본_2021.01.18.pdf
2        20408405_박성룡_초본_2020.11.13.pdf
3        20411188_임종우_초본_2021.05.07.pdf
4        20411194_김정하_초본_2020.10.21.pdf
                      ...              
72412            20496704_박수묵_양도통지서.pdf
72413      S28BW-5e22051915400_0027.pdf
72414      S28BW-5e22051915400_0028.pdf
72415                  베리타스_양수도_계약서.pdf
72416                베리타스양수도계약서-축약본.pdf
Name: 파일명, Length: 72417, dtype: object

In [ ]:
p = p1.match('1234')
print(p)

if p != None : 
    print(p.group())
    print(p.start())
    print(p.end())
    print(p.span())

m = re.search('^\d', '1234')
print(m)
if m != None : 
    print(m.group())
    print(m.start())
    print(m.end())
    print(m.span())

